# Data download

In [2]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 1616, done.
remote: Total 1616 (delta 0), reused 0 (delta 0), pack-reused 1616
Receiving objects: 100% (1616/1616), 219.51 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (774/774), done.
Checking out files: 100% (350/350), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cp -r drive/My\ Drive/normal ./

In [8]:
import os, random, cv2, shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Dropout,Flatten, AveragePooling2D, GlobalAveragePooling2D, Concatenate
import keras.optimizers as optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import plot_model

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
keras.__version__

Using TensorFlow backend.


'2.3.1'

# Overview

In [0]:
metadata = pd.read_csv("covid-chestxray-dataset/metadata.csv")

In [11]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               354 non-null    int64  
 1   offset                  258 non-null    float64
 2   sex                     311 non-null    object 
 3   age                     300 non-null    float64
 4   finding                 354 non-null    object 
 5   survival                109 non-null    object 
 6   intubated               71 non-null     object 
 7   intubation_present      61 non-null     object 
 8   went_icu                24 non-null     object 
 9   needed_supplemental_O2  6 non-null      object 
 10  extubated               16 non-null     object 
 11  temperature             28 non-null     float64
 12  pO2_saturation          37 non-null     float64
 13  leukocyte_count         9 non-null      float64
 14  neutrophil_count        2 non-null      fl

# Data preperation 

In [0]:
data_info = metadata.dropna(axis=1)

In [0]:
data_info = data_info.drop(['url'],axis=1)

In [0]:
data = data_info.drop(data_info[data_info['modality']=='CT'].index, axis=0)

In [0]:
final_data = data_info[data_info['view']=='PA']

In [0]:
final_data = final_data[final_data['finding']=='COVID-19']

In [17]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 353
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patientid  142 non-null    int64 
 1   finding    142 non-null    object
 2   view       142 non-null    object
 3   modality   142 non-null    object
 4   folder     142 non-null    object
 5   filename   142 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.8+ KB




---


**I have considered the COVID-19 PA view X-ray scans for my analysis**


---







In [0]:
positive_covid =final_data.drop(['view'],axis=1)
positive_covid =positive_covid.drop(['modality'],axis=1)
positive_covid =positive_covid.drop(['finding'],axis=1)
positive_covid =positive_covid.drop(['folder'],axis=1)

In [19]:
positive_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142 entries, 0 to 353
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patientid  142 non-null    int64 
 1   filename   142 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.3+ KB


In [20]:
positive_covid.head()

,patientid,filename
0,2,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1,2,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2,2,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3,2,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4,4,nejmc2001573_f1a.jpeg


In [0]:
listtt = positive_covid['filename']

In [22]:
len(positive_covid)

142

In [23]:
listtt

0      auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1      auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2      auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3      auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4                                  nejmc2001573_f1a.jpeg
                             ...                        
349                                  7-fatal-covid19.jpg
350                                     extubation-1.jpg
351                                     extubation-4.jpg
352                                     extubation-8.jpg
353                                    extubation-13.jpg
Name: filename, Length: 142, dtype: object

And now we are going to use utility function to parse COVID-19 images into training forlder 

In [0]:
def parse_covid(metadata, img_dir, target_dir='covid'):
  print("Start parsing the images from {} directory".format(img_dir))
  img_files= metadata['filename']
  if not os.path.exists(target_dir):
    os.mkdir(target_dir)
  for img, idx in zip(img_files, range(len(metadata))):
    src = os.path.join(img_dir,img)
    dst = os.path.join(target_dir,str(idx)+'.png')
    shutil.copyfile(src, dst)
  print('Done copying {0} images from {1} directory'.format(len(metadata), img_dir))

In [25]:
parse_covid(positive_covid, 'covid-chestxray-dataset/images')

Start parsing the images from covid-chestxray-dataset/images directory
Done copying 142 images from covid-chestxray-dataset/images directory


**I will sample 120 images of normal patients from RSNA Pneumonia dataset**

So.. 

`Normal`: 120 images
`COVID-19`: 142 images 

# Data preprocessing

In [0]:
class Train_val_split:
  def __init__(self, covid_dir='covid', normal_dir='normal', split_size=0.15):
    self.covid_dir = covid_dir
    self.normal_dir = normal_dir
    self.split_size = split_size
    self.train_dir = 'train'
    self.val_dir = 'validation'
    if not os.path.exists(self.train_dir):
      os.mkdir(self.train_dir)
    if not os.path.exists(self.val_dir):
      os.mkdir(self.val_dir)

  def split_class(self,target_class):
    if not os.path.exists(self.train_dir+'/'+target_class):
      os.mkdir(self.train_dir+'/'+target_class)
    if not os.path.exists(self.val_dir+'/'+target_class):
      os.mkdir(self.val_dir+'/'+target_class)
    
    data_indices = list(range(len(os.listdir(target_class))))
    random.shuffle(training_indices) 
    split_size = (1-self.split_size)*len(training_indices)
    training_idx = training_indices[:int(self.split_size)]
    validation_idx = training_indices[int(self.split_size):]

    for idx in training_idx:
      src = os.path.join(target_class, str(idx)+'.png')
      dst = os.path.join(self.train_dir+'/'+target_class, str(idx)+'.png')
      shutil.copyfile(src, dst)
    
    for idx in validation_idx:
      src = os.path.join(target_class, str(idx)+'.png')
      dst = os.path.join(self.val_dir+'/'+target_class, str(idx)+'.png')
      shutil.copyfile(src, dst)

In [0]:
split_data = Train_val_split('covid','normal',0.15)

In [0]:
split_data.split_class('covid')
split_data.split_class('normal')

# Data loaders

# Classification Model

#Model Evaluation

# References